<a href="https://colab.research.google.com/github/ahatasham5/Bangla_Handwritting_CSE366/blob/main/Trying_to_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
"""## 1. Import Required Libraries"""

import os
import zipfile
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from IPython.display import display, HTML, Javascript
import ipywidgets as widgets
from google.colab import drive, output

In [14]:
"""## 2. Mount Drive and Extract Dataset"""

# Mount Google Drive
drive.mount('/content/drive')

# Function to extract dataset
def extract_dataset(zip_path, extract_path='/content'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Dataset extracted to {extract_path}")

# Replace with your zip file path
ZIP_PATH = '/content/drive/MyDrive/HWrec/archive3.zip'
extract_dataset(ZIP_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted to /content


In [15]:
"""## 3. Define Model Architecture"""

def create_model(input_shape=(40, 40, 3), num_classes=10):
    model = Sequential([
        # First Convolutional Block
        Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Dropout(0.2),

        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.2),

        # Third Convolutional Block
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.2),

        # Flatten layer
        Flatten(),

        # Dense layers
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [16]:
"""## 4. Data Preparation"""

def setup_data_generators(train_path, test_path, target_size=(40, 40), batch_size=32):
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        fill_mode='nearest'
    )

    # Validation/Test data generator
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Create generators
    training_set = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_set = test_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    return training_set, test_set

In [17]:
"""## 5. Model Training"""

def train_model(model, training_set, test_set, epochs=50):
    # Define callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    checkpoint = ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    # Train the model
    history = model.fit(
        training_set,
        validation_data=test_set,
        epochs=epochs,
        callbacks=[early_stopping, checkpoint]
    )

    return history


In [18]:
"""## 6. Canvas Implementation"""

canvas_html = """
<div style="text-align: center;">
    <canvas width="400" height="400" id="drawing-canvas" style="border:2px solid black; background-color: white;"></canvas>
</div>
"""

canvas_js = """
<script>
    var canvas = document.getElementById('drawing-canvas');
    var ctx = canvas.getContext('2d');

    // Initial setup
    ctx.fillStyle = 'white';
    ctx.fillRect(0, 0, canvas.width, canvas.height);
    ctx.strokeStyle = 'black';
    ctx.lineWidth = 15;
    ctx.lineCap = 'round';
    ctx.lineJoin = 'round';

    var isDrawing = false;
    var lastX = 0;
    var lastY = 0;

    function draw(e) {
        if (!isDrawing) return;

        ctx.beginPath();
        ctx.moveTo(lastX, lastY);
        ctx.lineTo(e.offsetX, e.offsetY);
        ctx.stroke();
        [lastX, lastY] = [e.offsetX, e.offsetY];
    }

    canvas.addEventListener('mousedown', (e) => {
        isDrawing = true;
        [lastX, lastY] = [e.offsetX, e.offsetY];
    });

    canvas.addEventListener('mousemove', draw);
    canvas.addEventListener('mouseup', () => isDrawing = false);
    canvas.addEventListener('mouseout', () => isDrawing = false);

    window.clearCanvas = function() {
        ctx.fillStyle = 'white';
        ctx.fillRect(0, 0, canvas.width, canvas.height);
        ctx.strokeStyle = 'black';
    }
</script>
"""


In [19]:
"""## 7. Prediction Functions"""
def determine_character(predicted_class):
    mapping = {
        0: 'বন্ধ(০)',
        1: 'এক(১)',
        2: 'দুই(২)',
        3: 'তিন(৩)',
        4: 'চার(৪)',
        5: 'পাঁচ(৫)',
        6: 'ছয়(৬)',
        7: 'সাত(৭)',
        8: 'আট(৮)',
        9: 'নয়(৯)'
    }
    return mapping.get(predicted_class, 'Unknown')

def preprocess_canvas_image(img_array):
    try:
        # Convert RGBA to RGB
        rgb_img = Image.fromarray(img_array, 'RGBA').convert('RGB')

        # Debug: Save the initial converted image
        rgb_img.save('debug_input.png')

        # Resize to model input size (center crop if needed)
        resized_img = rgb_img.resize((40, 40), Image.LANCZOS)

        # Debug: Save the resized image
        resized_img.save('debug_resized.png')

        # Convert to array and normalize
        img_array = img_to_array(resized_img)
        img_array = img_array / 255.0

        # Debug: Print array statistics
        print(f"Preprocessing Debug:")
        print(f"Input image shape: {img_array.shape}")
        print(f"Pixel value range: {img_array.min():.4f} - {img_array.max():.4f}")
        print(f"Mean pixel value: {img_array.mean():.4f}")

        # Add batch dimension
        img_array = np.expand_dims(img_array, axis=0)

        return img_array

    except Exception as e:
        print(f"Preprocessing error: {str(e)}")
        return None



def predict_from_canvas(model):
    # Define the JavaScript to get image data
    get_image_js = """
    async function getImageData() {
        const canvas = document.getElementById('drawing-canvas');
        const ctx = canvas.getContext('2d');
        const imgData = ctx.getImageData(0, 0, canvas.width, canvas.height);
        return {
            data: Array.from(imgData.data),
            width: canvas.width,
            height: canvas.height
        };
    }
    getImageData();
    """

    try:
        # Get data from JavaScript
        data = output.eval_js(get_image_js)

        # Convert to numpy array
        img_array = np.array(data['data'], dtype=np.uint8).reshape((data['height'], data['width'], 4))

        # Preprocess image
        processed_img = preprocess_canvas_image(img_array)

        if processed_img is None:
            print("Image preprocessing failed")
            return None, None, None

        # Make prediction
        prediction = model.predict(processed_img, verbose=1)  # Set verbose to 1 for more info

        # Detailed prediction logging
        print("\nDetailed Prediction Analysis:")
        for i, prob in enumerate(prediction[0]):
            print(f"Class {i} ({determine_character(i)}): {prob:.4f}")

        predicted_class = np.argmax(prediction[0])
        confidence = prediction[0][predicted_class]

        # Get Bengali character representation
        predicted_char = determine_character(predicted_class)

        return predicted_class, confidence, predicted_char

    except Exception as e:
        print(f"Prediction error: {str(e)}")
        return None, None, None

In [20]:
"""## 8. UI Setup"""

def setup_interface(model):
    # Display canvas
    display(HTML(canvas_html + canvas_js))

    # Create widgets
    predict_button = widgets.Button(
        description='Predict',
        button_style='primary',
        layout=widgets.Layout(width='100px')
    )

    clear_button = widgets.Button(
        description='Clear',
        button_style='warning',
        layout=widgets.Layout(width='100px')
    )

    output_label = widgets.Label(
        value='Draw a digit and click Predict',
        layout=widgets.Layout(margin='10px 0px')
    )

    # Debug output
    debug_output = widgets.Output()


    # Define button callbacks
    def on_predict_click(b):
        with debug_output:
            digit, conf, char = predict_from_canvas(model)
            if digit is not None:
                output_label.value = f'Predicted: {char} (Confidence: {conf:.2%})'
                print(f"\nPrediction details:")
                print(f"Numeric class: {digit}")
                print(f"Bengali character: {char}")
                print(f"Confidence: {conf:.2%}")
            else:
                output_label.value = 'Error occurred during prediction'

    def on_clear_click(b):
        output.eval_js('clearCanvas()')
        output_label.value = 'Draw a digit and click Predict'
        with debug_output:
            print("\nCanvas cleared")

    # Register callbacks
    predict_button.on_click(on_predict_click)
    clear_button.on_click(on_clear_click)

    # Display widgets
    buttons = widgets.HBox([predict_button, clear_button])
    display(buttons, output_label, debug_output)


In [21]:
"""## 9. Main Execution"""

def main():
    # Define paths
    TRAIN_PATH = '/content/archive3/Test'
    TEST_PATH = '/content/archive3/Train'
    MODEL_PATH = '/content/drive/MyDrive/bangla_model.keras'

    # Create and train model if it doesn't exist
    if not os.path.exists(MODEL_PATH):
        print("Training new model...")

        # Setup data
        training_set, test_set = setup_data_generators(TRAIN_PATH, TEST_PATH)

        # Create and train model
        model = create_model()
        history = train_model(model, training_set, test_set)

        # Save model
        model.save(MODEL_PATH)
        print(f"Model saved to {MODEL_PATH}")
    else:
        print("Loading existing model...")
        model = load_model(MODEL_PATH)

    # Setup prediction interface
    setup_interface(model)


In [22]:
"""## 10. Run the Application"""

if __name__ == "__main__":
    main()

Loading existing model...


Label(value='Draw a digit and click Predict', layout=Layout(margin='10px 0px'))

Output()